In [16]:
import pickle
from pathlib import Path
import sys

import pandas as pd
from sklearn.metrics import accuracy_score

rank = 0 
#sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, df_copy, train_or_test
from kayano import age_categolize, stSlope_categolize, cholesterol_mean

### データセットの用意

In [17]:
df = pd.read_csv('./data/train.csv')
df_x = df.iloc[:, :-1]
df_y = df.iloc[:, -1]

### パイプライン候補の準備

In [18]:
def pipe_1(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

def pipe_2(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = df_copy(pipe.df_num, stSlope_categolize(pipe.df_cat), 'ST_Slope')
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

def pipe_3(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = cholesterol_mean(pipe.df_num)
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

### データセットの作成

In [19]:
split_kwrg = {}
train_flg = False

# パイプライをセット
pipe_lines = [pipe_1, pipe_2, pipe_3]
data_set = {pipe.__name__: pipe(df_x, train_flg, split_kwrg).values for pipe in pipe_lines}

### モデルのロード

In [20]:
with open('./retrained_model.pkl', 'rb') as f:
    train_models = pickle.load(f)

### モデルの評価と選定

In [21]:
def test_eval(train_models, pipe_lines):
    predicts = {}
    for key in train_models.keys():
        model = train_models[key]
        scores = []
        for pipe in pipe_lines:
            x = data_set[pipe.__name__]
            pred = model[pipe.__name__].predict(x)
            score = accuracy_score(df_y.values, pred)
            scores.append(score)
        predicts[key] = scores
    return pd.DataFrame(predicts, index=[pipe.__name__ for pipe in pipe_lines])

In [22]:
evaluations = test_eval(train_models, pipe_lines)
evaluations.sort_values('XGBClassifier', ascending=False)

,XGBClassifier,SVC
pipe_3,0.978193,0.690031
pipe_1,0.937695,0.741433
pipe_2,0.912773,0.742991
